# Jupyter Notebook für Test Cases

In [1]:
%run ../../Setup.ipynb

import unittest
import subprocess
import pymongo
import bson
from testfunctions_notebook import cuboid_volume_notebook, division_notebook, division2_notebook
from bdcc.database.connection.database_connector import DatabaseConnector, get_matches

In [3]:
class TestGetMatches(unittest.TestCase):

    def test_get_all_matches(self):
        test_list = get_matches()
        test_call = get_matches()
        self.assertListEqual(test_call, test_list)
        self.assertEqual(test_call, test_list)
        self.assertEqual(len(test_call), len(test_list))

    def test_get_one_match_index_0(self):
        test_list = get_matches(num_matches=1)
        test_call = get_matches(num_matches=1)
        self.assertListEqual(test_call, test_list)
        self.assertEqual(test_call, test_list)
        self.assertEqual(len(test_call), 1)
    
    def test_get_one_match_index_10(self):
        test_list = get_matches(start_id=10, num_matches=1)
        test_call = get_matches(start_id=10, num_matches=1)
        self.assertListEqual(test_call, test_list)
        self.assertEqual(test_call, test_list)
        self.assertEqual(len(test_call), 1)
    
    def test_get_10_matches(self):
        test_list = get_matches(num_matches=10)
        test_call = get_matches(num_matches=10)
        self.assertListEqual(test_call, test_list)
        self.assertEqual(test_call, test_list)
        self.assertEqual(len(test_call), 10)
    
    def test_get_matches_return_type(self):
        test_call = get_matches(num_matches=1)
        self.assertIsInstance(test_call, list)
        self.assertIsInstance(test_call[0], dict)

In [4]:
class TestDatabaseConnector(unittest.TestCase):
    def setUp(self):
        self.db_con = DatabaseConnector()
    
    def test_DataBaseConnector_init(self):
        # url
        self.assertEqual(self.db_con.url, 'localhost')
        self.db_con.url = '127.0.0.1'
        self.assertEqual(self.db_con.url, '127.0.0.1')
        # port
        self.assertEqual(self.db_con.port, 27018)
        self.db_con.port = 27017
        self.assertEqual(self.db_con.port, 27017)
        # database
        self.assertEqual(self.db_con.database, 'Data-KI')
        self.db_con.database = 'Data'
        self.assertEqual(self.db_con.database, 'Data')
        # collection
        self.assertEqual(self.db_con.collection, 'MatchesDto')
        self.db_con.collection = 'Matches'
        self.assertEqual(self.db_con.collection, 'Matches')

    def test_DataBaseConnector_connect(self):
        test_call = self.db_con.connect()
        self.assertIsInstance(test_call, type(self.db_con))
        self.assertIsInstance(test_call.client, pymongo.MongoClient)
        self.assertIsInstance(test_call.db, pymongo.database.Database)
        self.assertIsInstance(test_call.connection, type(self.db_con))
        self.assertIsInstance(test_call.active_collection, pymongo.collection.Collection)
    
    def test_DataBaseConnector_disconnect(self):
        # client nicht verbunden
        self.assertRaises(ConnectionError, lambda: self.db_con.disconnect())
        test_call = self.db_con.connect()
        self.assertIsInstance(test_call, type(self.db_con))
    
    def test_DataBaseConnector_getCollectionNames(self):
        self.db_con.connect()
        test_call = self.db_con.getCollectionNames()
        self.assertIsInstance(test_call, list)

    def test_DataBaseConnector_getCollection(self):
        self.db_con.connect()
        test_call = self.db_con.getCollection('MatchesDto')
        self.assertIsInstance(test_call, pymongo.collection.Collection)

    def test_DataBaseConnector_create(self):
        self.db_con.connect()
        # TBD
        #test_call = self.db_con.create("{\"test_id\": 1, \"test_val\": 10")
        #self.assertIsInstance(test_call, bson.objectid.ObjectId)
    
    def test_DataBaseConnector_get(self):
        self.db_con.connect()
        test_call_all = self.db_con.get()
        test_call_one = self.db_con.get(id=1)
        self.assertIsInstance(test_call_all, pymongo.cursor.Cursor)
        self.assertGreaterEqual(len(list(test_call_all)), 1)
        self.assertIsInstance(test_call_one, pymongo.cursor.Cursor)
        self.assertEqual(len(list(test_call_one)), 1)
    
    def test_DataBaseConnector_update(self):
        pass

    def test_DataBaseConnector_remove(self):
        pass

In [5]:
def suite():
    """Funktion zum Erstellen einer Test Suite für die Ausführung der Unittests."""
    suite = unittest.TestSuite()
    suite.addTest(unittest.makeSuite(TestGetMatches, 'test'))
    suite.addTest(unittest.makeSuite(TestDatabaseConnector, 'test'))
    return suite

In [6]:
def execute_unittests():
    """Funktion zum Ausführen der Unittests."""
    unittest.main(argv=[''], verbosity=2, exit=False, defaultTest='suite')

In [7]:
def execute_coverage():
    # TBD
    pass
    # os.system("coverage run -m testcases.py")
    # subprocess.run(["coverage", "run", "-m", "testcases.py"])
    # result = subprocess.run(["coverage", "report", "-m"], stdout=subprocess.PIPE)
    # output = result.stdout
    # output = output.decode("utf-8")
    # print(output)
    # subprocess.run(["coverage", "html", "-d", "html"])
    # cov = coverage.Coverage()
    # cov.start()
    # unittest.main(argv=[''], verbosity=2, exit=False, defaultTest='suite')
    # cov.stop()
    # cov.save()
    # cov.html_report()